In [12]:
import sys
sys.path.insert(0, '/home/johnr_000/jupyter_blog/jupyter_blog/scripts')
import eia_model as em
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.cross_validation import train_test_split
from datetime import datetime

In [19]:
#variables to specify date range and frequency
api_key = em.eia_api
start = '2015-12-01 01:00:00'
end = '2016-01-31 23:00:00'
freq = 'H'
# create list of series used for dictionary keys
keys = ['EBA.BPAT-ALL.D.H', 'EBA.PACE-ALL.D.H', 'EBA.PACW-ALL.D.H', 
        'EBA.PGE-ALL.D.H', 'EBA.PSEI-ALL.D.H', 'EBA.SCL-ALL.D.H']
# create dict of GetSeriesRange objects from series list
series_dict = {key: em.GetSeries(api_key=api_key, series_id=key, 
                                   start=start, end=end, freq=freq) for key in keys}

In [20]:
# loop over dictionary and print out full name of each series
for key in series_dict:
    print('{}: {}'.format(series_dict[key].data.series[0]['name'],key))

Demand for PacifiCorp West (PACW), Hourly: EBA.PACW-ALL.D.H
Demand for Seattle City Light (SCL), Hourly: EBA.SCL-ALL.D.H
Demand for Portland General Electric Company (PGE), Hourly: EBA.PGE-ALL.D.H
Demand for Puget Sound Energy, Inc. (PSEI), Hourly: EBA.PSEI-ALL.D.H
Demand for Bonneville Power Administration (BPAT), Hourly: EBA.BPAT-ALL.D.H
Demand for PacifiCorp East (PACE), Hourly: EBA.PACE-ALL.D.H


In [21]:
# all the dataframes have the same date range so we can concatenate in a loop
concat_df = pd.concat([series_dict[key].data.df for key in series_dict], axis=1)
# and output as a pickle file - I'm doing this to avoid calling the EIA API as I develop
concat_df.to_pickle('data/nw_load_df.pkl')
# concat_df = pd.read_pickle('data/new_load_df.pkl')

In [22]:
concat_df.describe()

/home/johnr_000/venv_python3/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,EBA.PACW-ALL.D.H,EBA.SCL-ALL.D.H,EBA.PGE-ALL.D.H,EBA.PSEI-ALL.D.H,EBA.BPAT-ALL.D.H,EBA.PACE-ALL.D.H
count,1.487000e+03,1487.000000,1487.000000,1482.000000,1487.000000,1.487000e+03
mean,4.451198e+03,1282.258911,2579.921991,3946.066127,7059.477471,-2.470617e+04
std,6.558528e+04,177.394473,385.764476,554.268601,833.190306,8.950455e+05
min,-6.785000e+03,868.000000,1730.000000,2292.000000,5006.000000,-3.199017e+07
25%,2.393500e+03,1128.500000,2272.000000,NaN,6451.500000,5.169000e+03
50%,2.709000e+03,1324.000000,2646.000000,NaN,7109.000000,5.508000e+03
75%,2.941500e+03,1416.000000,2872.500000,NaN,7637.500000,5.822500e+03
max,2.530003e+06,1654.000000,3472.000000,5088.000000,9365.000000,5.966800e+04


In [23]:
train, test = train_test_split(concat_df, test_size = 0.2)
print('Length of training dataframe: {} and testing dataframe {}'.format(len(train), len(test)))

Length of training dataframe: 1189 and testing dataframe 298


In [25]:
rng = np.random.RandomState(42)
clf = IsolationForest(max_samples=100, random_state=rng)
clf.fit(train['EBA.SCL-ALL.D.H'])
y_pred_train = clf.predict(train['EBA.SCL-ALL.D.H'])
y_pred_test = clf.predict(test['EBA.SCL-ALL.D.H'])

/home/johnr_000/venv_python3/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found input variables with inconsistent numbers of samples: [1, 1189]

In [26]:
train['EBA.SCL-ALL.D.H']

2015-12-27 20:00:00    1367
2015-12-21 19:00:00    1524
2015-12-07 19:00:00    1417
2015-12-06 13:00:00     966
2016-01-21 17:00:00    1410
2016-01-26 02:00:00    1477
2015-12-09 13:00:00     920
2015-12-30 14:00:00    1264
2015-12-08 01:00:00    1435
2016-01-06 22:00:00    1316
2015-12-06 12:00:00     962
2016-01-02 03:00:00    1450
2015-12-20 14:00:00    1056
2015-12-23 00:00:00    1445
2015-12-16 01:00:00    1535
2015-12-29 04:00:00    1456
2016-01-24 09:00:00    1039
2016-01-18 10:00:00     996
2015-12-10 04:00:00    1356
2016-01-13 07:00:00    1217
2015-12-01 12:00:00    1067
2015-12-04 19:00:00    1363
2015-12-28 12:00:00    1052
2016-01-25 02:00:00    1343
2016-01-18 17:00:00    1380
2016-01-09 03:00:00    1503
2015-12-24 06:00:00    1357
2016-01-28 07:00:00    1112
2016-01-09 08:00:00    1213
2016-01-06 20:00:00    1376
                       ... 
2015-12-22 19:00:00    1517
2016-01-03 12:00:00    1090
2016-01-29 03:00:00    1338
2016-01-01 14:00:00    1212
2015-12-21 07:00:00 